In [27]:
from syft import Worker, MessageStatus
from syft.core.node.new.project import Project
import syft as sy

In [28]:
worker = Worker.named("test-domain-1", processes=1, reset=False)

SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Starting Worker: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.Data

In [29]:
domain_client = worker.guest_client.login(email="info@openmined.org", password="changethis")

# Get messages

In [30]:
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)

In [31]:
messages

,type,id,subject,status,created_at,linked_obj
0,syft.core.node.new.messages.Message,c918e282d3de4e67b23f04cf937ec6b4,Project Approval,MessageStatus.UNDELIVERED,2023-04-17 10:48:08,<<class 'syft.core.node.new.project.Project'>:...
1,syft.core.node.new.messages.Message,200feb80573b40cc96f2171226bcf717,Approval Request,MessageStatus.UNDELIVERED,2023-04-17 10:48:05,<<class 'syft.core.node.new.request.Request'>:...


In [32]:
project_message = [x for x in messages if issubclass(x.linked_obj.object_type, Project)][0]

In [33]:
request = project_message.link.requests[0]
func = request.changes[0].link
op = func.output_policy_type

In [34]:
func

```python
class UserCode:
  id: str = 4280ac82d1464437a39245c1b5f24b37
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  user_verify_key: str = dd2e52e5808c53b643a7be2fce41123c8c34a7547cf34b50e3db9cb3e51d201f
  raw_code: str = "@sy.syft_function(input_policy=sy.ExactMatch(df=asset.mock),
                  output_policy=sy.SingleExecutionExactOutput())
def get_counts(df):
    complaint_counts = df['Complaint Type'].value_counts()
    return complaint_counts[:10]
"
  input_policy_type: str = <class 'syft.core.node.new.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'df': <UID: fdaa29c62b85426581d6beaddef185b7>}}
  input_policy_state: str = b''
  output_policy_type: str = <class 'syft.core.node.new.policy.OutputPolicyExecuteOnce'>
  output_policy_init_kwargs: str = {}
  output_policy_state: str = b''
  parsed_code: str = "

def user_func_get_counts_dd2e52e5808c53b643a7be2fce41123c8c34a7547cf34b50e3db9cb3e51d201f_ced68a4f95bc589adb5d44379eadb950ec506fd867517162b08636312d78c3f6(df):

    def get_counts(df):
        complaint_counts = df['Complaint Type'].value_counts()
        return complaint_counts[:10]
    result = get_counts(df=df)
    return result
"
  service_func_name: str = "get_counts"
  unique_func_name: str = "user_func_get_counts_dd2e52e5808c53b643a7be2fce41123c8c34a7547cf34b50e3db9cb3e51d201f_ced68a4f95bc589adb5d44379eadb950ec506fd867517162b08636312d78c3f6"
  user_unique_func_name: str = "user_func_get_counts_dd2e52e5808c53b643a7be2fce41123c8c34a7547cf34b50e3db9cb3e51d201f"
  code_hash: str = "ced68a4f95bc589adb5d44379eadb950ec506fd867517162b08636312d78c3f6"
  signature: str = (df)
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['df']
  enclave_metadata: str = None

```

In [35]:
get_counts_user_func = func.unsafe_function

In [36]:
real_data = domain_client.datasets[0].assets[0].data

In [37]:
real_result = get_counts_user_func(df=real_data)

In [38]:
real_result[:3]

HEATING                   14200
GENERAL CONSTRUCTION       7471
Street Light Condition     7117
Name: Complaint Type, dtype: int64

In [39]:
result = request.accept_by_depositing_result(real_result)
result
assert isinstance(result, sy.SyftSuccess)